

# Configuring MNE-Python

This tutorial covers how to configure MNE-Python to suit your local system and
your analysis preferences.

We begin by importing the necessary Python modules:


In [1]:
# Authors: The MNE-Python contributors.
# License: BSD-3-Clause
# Copyright the MNE-Python contributors.

In [2]:
import os

import mne
from mne.datasets import testing


## Getting and setting configuration variables

Configuration variables are read and written using the functions
:func:`mne.get_config` and :func:`mne.set_config`. To read a specific
configuration variable, pass its name to :func:`~mne.get_config` as the
``key`` parameter (``key`` is the first parameter so you can pass it unnamed
if you want):



In [3]:
print(mne.get_config("MNE_USE_CUDA"))
print(type(mne.get_config("MNE_USE_CUDA")))

None
<class 'NoneType'>


Note that the string values read from the JSON file are not parsed in any
way, so :func:`~mne.get_config` returns a string even for true/false config
values, rather than a Python `boolean <python:typebool>`.
Similarly, :func:`~mne.set_config` will only set string values (or ``None``
values, to unset a variable):



In [4]:
try:
    mne.set_config("MNE_USE_CUDA", True)
except TypeError as err:
    print(err)

value must be an instance of str, path-like, or NoneType, got <class 'bool'> instead.


If you're unsure whether a config variable has been set, there is a
convenient way to check it and provide a fallback in case it doesn't exist:
:func:`~mne.get_config` has a ``default`` parameter.



In [5]:
print(mne.get_config("missing_config_key", default="fallback value"))

fallback value


There are also two convenience modes of :func:`~mne.get_config`. The first
will return a :class:`dict` containing all config variables (and their
values) that have been set on your system; this is done by passing
``key=None`` (which is the default, so it can be omitted):



In [6]:
print(mne.get_config())  # same as mne.get_config(key=None)

{'MNE_DATASETS_SAMPLE_PATH': '/home/jshen/mne_data', 'MNE_DATASETS_SSVEP_PATH': '/home/jshen/mne_data', 'MNE_DATASETS_TESTING_PATH': '/home/jshen/mne_data'}


The second convenience mode will return a :class:`tuple` of all the keys that
MNE-Python recognizes and uses, regardless of whether they've been set on
your system. This is done by passing an empty string ``''`` as the ``key``:



In [7]:
print(mne.get_config(key=""))

{'MNE_3D_OPTION_ANTIALIAS': 'bool, whether to use full-screen antialiasing in 3D plots', 'MNE_3D_OPTION_DEPTH_PEELING': 'bool, whether to use depth peeling in 3D plots', 'MNE_3D_OPTION_MULTI_SAMPLES': 'int, number of samples to use for full-screen antialiasing', 'MNE_3D_OPTION_SMOOTH_SHADING': 'bool, whether to use smooth shading in 3D plots', 'MNE_3D_OPTION_THEME': 'str, the color theme (light or dark) to use for 3D plots', 'MNE_BROWSE_RAW_SIZE': 'tuple, width and height of the raw browser window (in inches)', 'MNE_BROWSER_BACKEND': 'str, the backend to use for the MNE Browse Raw window (qt or matplotlib)', 'MNE_BROWSER_OVERVIEW_MODE': 'str, the overview mode to use in the MNE Browse Raw window )(see mne.viz.plot_raw for valid options)', 'MNE_BROWSER_PRECOMPUTE': 'bool, whether to precompute raw data in the MNE Browse Raw window', 'MNE_BROWSER_THEME': 'str, the color theme (light or dark) to use for the browser', 'MNE_BROWSER_USE_OPENGL': 'bool, whether to use OpenGL for rendering in 

It is possible to add config variables that are not part of the recognized
list, by passing any arbitrary key to :func:`~mne.set_config`. This will
yield a warning, however, which is a nice check in cases where you meant to
set a valid key but simply misspelled it:



In [8]:
mne.set_config("MNEE_USE_CUUDAA", "false")

/tmp/ipykernel_211313/1056208348.py:1: RuntimeWarning: Setting non-standard config type: "MNEE_USE_CUUDAA"
  mne.set_config("MNEE_USE_CUUDAA", "false")


Let's delete that config variable we just created. To unset a config
variable, use :func:`~mne.set_config` with ``value=None``. Since we're still
dealing with an unrecognized key (as far as MNE-Python is concerned) we'll
still get a warning, but the key will be unset:



In [9]:
mne.set_config("MNEE_USE_CUUDAA", None)
assert "MNEE_USE_CUUDAA" not in mne.get_config("")

/tmp/ipykernel_211313/1027864612.py:1: RuntimeWarning: Setting non-standard config type: "MNEE_USE_CUUDAA"
  mne.set_config("MNEE_USE_CUUDAA", None)


## Where configurations are stored

MNE-Python stores configuration variables in a `JSON`_ file. By default, this
file is located in :file:`{%USERPROFILE%}\\.mne\\mne-python.json` on Windows
and :file:`{$HOME}/.mne/mne-python.json` on Linux or macOS. You can get the
full path to the config file with :func:`mne.get_config_path`.



In [10]:
print(mne.get_config_path())

/home/jshen/.mne/mne-python.json


However it is not a good idea to directly edit files in the :file:`.mne`
directory; use the getting and setting functions described in `the
previous section <config-get-set>`.

If for some reason you want to load the configuration from a different
location, you can pass the ``home_dir`` parameter to
:func:`~mne.get_config_path`, specifying the parent directory of the
:file:`.mne` directory where the configuration file you wish to load is
stored.


## Using environment variables

For compatibility with :doc:`MNE-C <../../install/mne_c>`, MNE-Python
also reads and writes `environment variables`_ to specify configuration. This
is done with the same functions that read and write the JSON configuration,
and is controlled with the parameters ``use_env`` and ``set_env``. By
default, :func:`~mne.get_config` will check :data:`os.environ` before
checking the MNE-Python JSON file; to check *only* the JSON file use
``use_env=False``. To demonstrate, here's an environment variable that is not
specific to MNE-Python (and thus is not in the JSON config file):



In [11]:
# make sure it's not in the JSON file (no error means our assertion held):
assert mne.get_config("PATH", use_env=False) is None
# but it *is* in the environment:
print(mne.get_config("PATH"))

/home/jshen/miniconda3/envs/Summer2025DL/bin:/home/jshen/.vscode-server/bin/cb0c47c0cfaad0757385834bd89d410c78a856c0/bin/remote-cli:/home/jshen/.local/bin:/home/jshen/miniconda3/condabin:/home/jshen/.nvm/versions/node/v23.9.0/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/wsl/lib:/mnt/c/Program Files/Common Files/Oracle/Java/javapath:/mnt/c/Program Files (x86)/Common Files/Oracle/Java/java8path:/mnt/c/Program Files (x86)/Common Files/Oracle/Java/javapath:/mnt/c/Program Files (x86)/Razer Chroma SDK/bin:/mnt/c/Program Files/Razer Chroma SDK/bin:/mnt/c/Program Files (x86)/Razer/ChromaBroadcast/bin:/mnt/c/Program Files/Razer/ChromaBroadcast/bin:/mnt/c/Windows/system32:/mnt/c/Windows:/mnt/c/Windows/System32/Wbem:/mnt/c/Windows/System32/WindowsPowerShell/v1.0:/mnt/c/Windows/System32/OpenSSH:/mnt/c/Program Files (x86)/NVIDIA Corporation/PhysX/Common:/mnt/c/WINDOWS/system32:/mnt/c/WINDOWS:/mnt/c/WINDOWS/System32/Wbem:/mnt/c/WINDOWS/System3

Also by default, :func:`~mne.set_config` will set values in both the JSON
file and in :data:`os.environ`; to set a config variable *only* in the JSON
file use ``set_env=False``. Here we'll use :func:`print` statement to confirm
that an environment variable is being created and deleted (we could have used
the Python `assert statement <assert>` instead, but it doesn't print any
output when it succeeds so it's a little less obvious):



In [12]:
mne.set_config("foo", "bar", set_env=False)
print("foo" in os.environ.keys())
mne.set_config("foo", "bar")
print("foo" in os.environ.keys())
mne.set_config("foo", None)  # unsetting a key deletes var from environment
print("foo" in os.environ.keys())

False
True
False


/tmp/ipykernel_211313/2731098602.py:1: RuntimeWarning: Setting non-standard config type: "foo"
  mne.set_config("foo", "bar", set_env=False)
/tmp/ipykernel_211313/2731098602.py:3: RuntimeWarning: Setting non-standard config type: "foo"
  mne.set_config("foo", "bar")
/tmp/ipykernel_211313/2731098602.py:5: RuntimeWarning: Setting non-standard config type: "foo"
  mne.set_config("foo", None)  # unsetting a key deletes var from environment



## Logging

One important configuration variable is ``MNE_LOGGING_LEVEL``. Throughout the
module, messages are generated describing the actions MNE-Python is taking
behind-the-scenes. How you set ``MNE_LOGGING_LEVEL`` determines how many of
those messages you see. The default logging level on a fresh install of
MNE-Python is ``info``:



In [13]:
print(mne.get_config("MNE_LOGGING_LEVEL"))

None


The logging levels that can be set as config variables are ``debug``,
``info``, ``warning``, ``error``, and ``critical``. Around 90% of the log
messages in MNE-Python are ``info`` messages, so for most users the choice is
between ``info`` (tell me what is happening) and ``warning`` (tell me only if
something worrisome happens). The ``debug`` logging level is intended for
MNE-Python developers.


In `an earlier section <config-get-set>` we saw how
:func:`mne.set_config` is used to change the logging level for the current
Python session and all future sessions. To change the logging level only for
the current Python session, you can use :func:`mne.set_log_level` instead.
The :func:`~mne.set_log_level` function takes the same five string options
that are used for the ``MNE_LOGGING_LEVEL`` config variable; additionally, it
can accept :class:`int` or :class:`bool` values that are equivalent to those
strings. The equivalencies are given in this table:

| String | Integer | Boolean |
|---|---|---|
| DEBUG | 10 | |
| INFO | 20 | True |
| WARNING | 30 | False |
| ERROR | 40 | |
| CRITICAL | 50 | |

With many MNE-Python functions it is possible to change the logging level
temporarily for just that function call, by using the ``verbose`` parameter.
To illustrate this, we'll load some sample data with different logging levels
set. First, with log level ``warning``:



In [14]:
kit_data_path = testing.data_path(verbose=True) / "KIT" / "test_as-raw.con"
raw = mne.io.read_raw_kit(kit_data_path, verbose="warning")

No messages were generated, because none of the messages were of severity
"warning" or worse. Next, we'll load the same file with log level ``info``
(the default level):



In [15]:
raw = mne.io.read_raw_kit(kit_data_path, verbose="info")

Extracting SQD Parameters from /home/jshen/mne_data/MNE-testing-data/KIT/test_as-raw.con...
Creating Raw.info structure...
Setting channel info structure...
Creating Info structure...
Ready.


This time, we got a few messages about extracting information from the file,
converting that information into the MNE-Python :class:`~mne.Info` format,
etc. Finally, if we request ``debug``-level information, we get even more
detail -- and we do so this time using the :func:`mne.use_log_level` context
manager, which is another way to accomplish the same thing as passing
``verbose='debug'``:



In [16]:
with mne.use_log_level("debug"):
    raw = mne.io.read_raw_kit(kit_data_path)

Extracting SQD Parameters from /home/jshen/mne_data/MNE-testing-data/KIT/test_as-raw.con...
Creating Raw.info structure...
    KIT dir entry 0 @ 16
    KIT dir entry 1 @ 32
    KIT dir entry 2 @ 48
    KIT dir entry 3 @ 64
    KIT dir entry 4 @ 80
    KIT dir entry 5 @ 96
    KIT dir entry 6 @ 112
    KIT dir entry 7 @ 128
    KIT dir entry 8 @ 144
    KIT dir entry 9 @ 160
    KIT dir entry 10 @ 176
    KIT dir entry 11 @ 192
    KIT dir entry 12 @ 208
    KIT dir entry 13 @ 224
    KIT dir entry 14 @ 240
    KIT dir entry 15 @ 256
    KIT dir entry 16 @ 272
    KIT dir entry 17 @ 288
    KIT dir entry 18 @ 304
    KIT dir entry 19 @ 320
    KIT dir entry 20 @ 336
    KIT dir entry 21 @ 352
    KIT dir entry 22 @ 368
    KIT dir entry 23 @ 384
    KIT dir entry 24 @ 400
    KIT dir entry 25 @ 416
    KIT dir entry 26 @ 432
    KIT dir entry 27 @ 448
    KIT dir entry 28 @ 464
    KIT dir entry 29 @ 480
    KIT dir entry 30 @ 496
SQD file basic information:
Meg160 version = V2R004
Syst

We've been passing string values to the ``verbose`` parameter, but we can see
from `the table above <table-log-levels>` that ``verbose=True`` will
give us the ``info`` messages and ``verbose=False`` will suppress them; this
is a useful shorthand to use in scripts, so you don't have to remember the
specific names of the different logging levels. One final note:
``verbose=None`` (which is the default for functions that have a ``verbose``
parameter) will fall back on whatever logging level was most recently set by
:func:`mne.set_log_level`, or if that hasn't been called during the current
Python session, it will fall back to the value of
``mne.get_config('MNE_LOGGING_LEVEL')``.

## Getting information about your system
You can also get information about what ``mne`` imports as dependencies from
your system. This can be done via the command line with:

```console
$ mne sys_info
```
Or you can use :func:`mne.sys_info` directly, which prints to ``stdout`` by
default:



In [17]:
mne.sys_info()

Platform             Linux-6.6.87.2-microsoft-standard-WSL2-x86_64-with-glibc2.35
Python               3.9.23 (main, Jun  5 2025, 13:40:20)  [GCC 11.2.0]
Executable           /home/jshen/miniconda3/envs/Summer2025DL/bin/python
CPU                  x86_64 (16 cores)
Memory               7.7 GB

Core
├☒ mne               1.8.0 (outdated, release 1.9.0 is available!)
├☑ numpy             2.0.1 (MKL 2023.1-Product with 8 threads)
├☑ scipy             1.13.1
└☑ matplotlib        3.9.2 (backend=module://matplotlib_inline.backend_inline)

Numerical (optional)
├☑ sklearn           1.6.1
├☑ nibabel           5.3.2
├☑ pandas            2.2.3
└☐ unavailable       numba, nilearn, dipy, openmeeg, cupy, h5io, h5py

Visualization (optional)
├☑ pyvista           0.45.0 (OpenGL 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.3 via llvmpipe (LLVM 15.0.7, 256 bits))
├☑ pyvistaqt         0.11.2
├☑ vtk               9.1.0
├☑ qtpy              2.4.3 (PyQt6=6.7.3)
├☑ ipywidgets        8.1.7
└☐ unavailable   

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


.. LINKS


